# Exercício 1

## Importação da tabela food_events dentro do dataset fda_food

In [3]:
from google.cloud import bigquery

In [7]:
## credenciais

project_id = "teste-greeenpeace"
json_path = "C:/Users/User/Desktop/greenpeace_teste/teste-greeenpeace-9d29e6d836fb.json"

client = bigquery.Client.from_service_account_json(json_path) 


In [12]:
dataset_id = "dataset_greenpeace" 

# objeto de referência
dataset_ref = bigquery.DatasetReference(project_id, dataset_id)

# objeto Dataset
dataset = bigquery.Dataset(dataset_ref)
dataset.friendly_name = "dataset_greenpeace"


client.create_dataset(dataset)

# Conexão
dataset = client.get_dataset(dataset_ref)

In [14]:
table_id = "food_events"

In [15]:
source_dataset_ref = bigquery.DatasetReference("bigquery-public-data", "fda_food")
source_table_ref = bigquery.TableReference(source_dataset_ref, table_id)

In [16]:
# tabela de destino
destination_dataset_ref = bigquery.DatasetReference(project_id, dataset_id)
destination_table_ref = bigquery.TableReference(destination_dataset_ref, table_id)

In [17]:
# tabela de origem  - tabela de destino
job_config = bigquery.CopyJobConfig()
job = client.copy_table(source_table_ref, destination_table_ref, job_config=job_config)
job.result() 

CopyJob<project=teste-greeenpeace, location=US, id=b7cd4eaa-2f6c-4b95-a0ca-b029d468faf4>

In [21]:
# Visualizar colunas para consultas

column_names = [field.name for field in table.schema]
print("Colunas:")
for column_name in column_names:
    print(column_name)

Colunas:
report_number
reactions
outcomes
products_brand_name
products_industry_code
products_role
products_industry_name
date_created
date_started
consumer_gender
consumer_age
consumer_age_unit


## Qual o tipo de reação mais reportado na tabela?

In [79]:
query = f"""
SELECT reactions, COUNT(*) AS count
FROM `{project_id}.{dataset_id}.{table_id}`
GROUP BY reactions
ORDER BY count DESC
LIMIT 1
"""

query_job = client.query(query)
results = query_job.result()


for row in results:
    most_reported_reaction = row['reactions']
    report_count = row['count']

# Imprimir o tipo de reação mais reportado
print(f"O mais reportado na tabela é '{most_reported_reaction}' com um total de {report_count}.")

O mais reportado na tabela é 'Ovarian cancer' com um total de 19316.


## Qual o nome da indústria que tem mais mortes reportadas?

In [58]:
industrias_mortes = """
SELECT 
    products_industry_name AS industry_name, 
    COUNT(*) AS count
FROM `{}.{}.{}` 
GROUP BY products_industry_name
ORDER BY count DESC
LIMIT 1
""".format(project_id, dataset_id, table_id)

query_job = client.query(industrias_mortes)
result_industry_with_most_deaths = query_job.result()

if result_industry_with_most_deaths:
    first_row = next(result_industry_with_most_deaths)
    industry_name = first_row["industry_name"]
    count = first_row["count"]
    print("Indústria com mais mortes reportadas: {} ({} mortes)".format(industry_name, count))
else:
    print("Nenhum resultado encontrado.")


Indústria com mais mortes reportadas: Vit/Min/Prot/Unconv Diet(Human/Animal) (82273 mortes)


## Quais as 3 principais reações adversas de pessoas entre 18-25 anos na indústria de cosméticos?

In [77]:
top_3_reacoes = f"""
SELECT reactions, COUNT(*) AS count
FROM `{project_id}.{dataset_id}.{table_id}`
WHERE consumer_age BETWEEN 18 AND 25 AND products_industry_name = 'Cosmetics'
GROUP BY reactions
ORDER BY count DESC
LIMIT 3
"""

# Executar a consulta e obter resultados
query_job = client.query(top_3_reacoes)
results = query_job.result()

# Imprimir as três principais reações adversas
print("As três principais reações adversas de pessoas entre 18-25 anos na indústria de cosméticos são:")
for row in results:
    print(f"Reação: {row['reactions']}, Contagem: {row['count']}")

As três principais reações adversas de pessoas entre 18-25 anos na indústria de cosméticos são:
Reação: Ovarian cancer, Contagem: 298
Reação: Alopecia, Contagem: 101
Reação: Ovarian cancer stage I, Contagem: 60
